In [1]:
from mnmt.encoder import BasicEncoder
from mnmt.decoder import BasicFeedForwardDecoder
from mnmt.decoder import GreedyDecoder
from mnmt.attention import AdditiveAttention
from mnmt.decoder import BridgeLayer
from mnmt.model import Seq2Seq, Seq2MultiSeq
from mnmt.datasets import *
from mnmt.inputter import ArgsFeeder
from mnmt.inputter import ModuleArgsFeeder
from mnmt.trainer.utils import *
from mnmt.trainer import Trainer
import torch.nn as nn
import sys

Loading DICT dataset...
Loading the Dataset into the container...
Field names: ['en', 'ch', 'pinyin_str', 'pinyin_char']
Data sizes: [(Train, 46620), (Valid, 5828), (Test, 5828)]
The first example of the training data is:
en :  ['k', 'a', 'r', 't', 'm', 'a', 'n']
ch :  ['卡', '特', '曼']
pinyin_str :  ['ka', 'te', 'man']
pinyin_char :  ['k', 'a', 't', 'e', 'm', 'a', 'n']
Loading NEWS dataset...
Loading the Dataset into the container...
Field names: ['en', 'ch', 'pinyin_str', 'pinyin_char']
Data sizes: [(Train, 81252), (Valid, 513), (Test, 1000)]
The first example of the training data is:
en :  ['k', 'u', 's', 'i', 'c', 'k']
ch :  ['库', '西', '克']
pinyin_str :  ['ku', 'xi', 'ke']
pinyin_char :  ['k', 'u', 'x', 'i', 'k', 'e']


In [5]:
def set_up_args(data_container, exp_num):
    build_vocabs(data_container, dict_min_freqs={'en': 1, 'ch': 1, 'pinyin_str': 1, 'pinyin_char': 1})
    for name, field in data_container.fields:
        if name == 'en':
            input_dim = len(field.vocab)
            src_pad_idx = field.vocab.stoi[field.pad_token]
        elif name == 'ch':
            output_dim = len(field.vocab)
            trg_pad_idx = field.vocab[field.pad_token]

    enc_args_feeder = ModuleArgsFeeder(input_dim=input_dim, embedding_dim=256, hidden_dim=512,
                                       embedding_dropout=0.1, rnn_type='LSTM',
                                       num_layers=2, rnn_dropout=0.2)
    dec_args_feeder = ModuleArgsFeeder(input_dim=output_dim, embedding_dim=256, hidden_dim=512,
                                       embedding_dropout=0.1, rnn_type='LSTM',
                                       num_layers=2, rnn_dropout=0.2)
    dec_args_feeder_aux = ModuleArgsFeeder(input_dim=output_dim, embedding_dim=128, hidden_dim=256,
                                       embedding_dropout=0.1, rnn_type='LSTM',
                                       num_layers=2, rnn_dropout=0.1)
    return ArgsFeeder(enc_args_feeder, [dec_args_feeder, dec_args_feeder_aux],
                      batch_size=64, src_pad_idx=src_pad_idx, trg_pad_idx=trg_pad_idx,
                      optim_choice='Adam', learning_rate=0.003, decay_patience=0,
                      lr_decay_factor=0.9, valid_criterion='ACC', early_stopping_patience=100,
                      total_epochs=100, report_interval=50, exp_num=exp_num, multi_task_ratio=66, data_container=data_container,
                      src_lang='en', trg_lang='ch', auxiliary_name='pinyin_str', quiet_translate=True,
                      valid_out_path=f"experiments/exp{exp_num}/valid.out", test_out_path=f"experiments/exp{exp_num}/test.out",
                      beam_size=1)


def test_seq2seq(args_feeder):
    
    decoder_args_feeder = args_feeder.decoder_args_feeders[0]
    decoder_args_feeder_aux = args_feeder.decoder_args_feeders[1]
    
    encoder = BasicEncoder(args_feeder)
    
    feed_forward_decoder = \
        BasicFeedForwardDecoder(args_feeder,
                                AdditiveAttention(encoder_hidden_dim=args_feeder.encoder_args_feeder.hidden_dim,
                                                  decoder_hidden_dim=decoder_args_feeder.hidden_dim), 
                                                  decoder_index=0)
    feed_forward_decoder_aux = \
            BasicFeedForwardDecoder(args_feeder,
                                    AdditiveAttention(encoder_hidden_dim=args_feeder.encoder_args_feeder.hidden_dim,
                                                      decoder_hidden_dim=decoder_args_feeder_aux.hidden_dim), 
                                                      decoder_index=1)
                                
    bridge_layer = BridgeLayer(encoder_hidden_dim=args_feeder.encoder_args_feeder.hidden_dim,
                               decoder_hidden_dim=decoder_args_feeder.hidden_dim,
                               num_of_states=2)
    bridge_layer_aux = BridgeLayer(encoder_hidden_dim=args_feeder.encoder_args_feeder.hidden_dim,
                                   decoder_hidden_dim=decoder_args_feeder_aux.hidden_dim,
                                   num_of_states=2)
                                
    decoder = GreedyDecoder(feed_forward_decoder, bridge_layer, device=args_feeder.device)
    decoder_aux = GreedyDecoder(feed_forward_decoder_aux, bridge_layer_aux, device=args_feeder.device)
                                
    model = Seq2MultiSeq(args_feeder, encoder, 
                         nn.ModuleList([decoder, decoder_aux]), 
                         teacher_forcing_ratio=0.8).to(args_feeder.device)
    return model


if __name__ == '__main__':
    set_reproducibility(seed=1234)
    mtrs = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
    aux_lang = "pinyin_char"  # pinyin_str, pinyin_char
    mtrs.reverse()
    try:
        # DICT Multi 
        dict_dataset = DICT['data_container']
        for i in range(6, len(mtrs)):
            i += 1
            seq2seq_args_feeder = set_up_args(dict_dataset, exp_num=20 + i)
            seq2seq_args_feeder.multi_task_ratio = mtrs[i-1]
            seq2seq_args_feeder.auxiliary_name = aux_lang
            test_model = test_seq2seq(seq2seq_args_feeder)
            test_trainer = Trainer(seq2seq_args_feeder, test_model)
            test_trainer.run(burn_in_epoch=0)
            test_trainer.best_model_output()
        # NEWS Multi
        news_dataset = NEWS['data_container']
        for i in range(len(mtrs)):
            i += 1
            seq2seq_args_feeder = set_up_args(news_dataset, exp_num=30 + i)
            seq2seq_args_feeder.multi_task_ratio = mtrs[i-1]
            seq2seq_args_feeder.auxiliary_name = aux_lang
            test_model = test_seq2seq(seq2seq_args_feeder)
            test_trainer = Trainer(seq2seq_args_feeder, test_model)
            test_trainer.run(burn_in_epoch=15)
            test_trainer.best_model_output(test_ref_dict=NEWS['test-set-dict'])
    except KeyboardInterrupt:
        print("Exiting loop")

The current device for PyTorch is cuda
Seq2MultiSeq(
  (encoder): BasicEncoder(
    (embedding): Sequential(
      (0): Embedding(30, 256)
      (1): Dropout(p=0.1, inplace=False)
    )
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.2, bidirectional=True)
  )
  (decoder_list): ModuleList(
    (0): GreedyDecoder(
      (feed_forward_decoder): BasicFeedForwardDecoder(
        (attention): AdditiveAttention(
          (additive_mapping): Linear(in_features=1536, out_features=512, bias=True)
          (v): Linear(in_features=512, out_features=1, bias=False)
        )
        (embedding): Sequential(
          (0): Embedding(437, 256)
          (1): Dropout(p=0.1, inplace=False)
        )
        (rnn): LSTM(1280, 512, num_layers=2, dropout=0.2)
        (prediction): Sequential(
          (0): Linear(in_features=1792, out_features=437, bias=True)
          (1): LogSoftmax()
        )
      )
      (bridge_layer): BridgeLayer(
        (bridge_layer): ModuleList(
          (0): Linear(in_f

[VAL]: The number of correct predictions (main-task (multi)): 3814/5828
[VAL]: The number of correct predictions (aux-task (multi)): 3782/5828

---------------------------------------
[Epoch: 2][Validatiing...]
		 Better Valid Loss! (at least equal)
		 Better Valid Acc! (at least equal)
	 Early Stopping Patience: 100/100
	 Val. Loss: 0.590 | Val. Acc: 0.654 | Val. PPL:   1.804
	 BEST. Val. Loss: 0.590 | BEST. Val. Acc: 0.654 | Val. Loss: 0.590 | BEST. Val. Epoch: 2 | BEST. Val. Step: 1500
---------------------------------------

		 Better Valid Acc on Auxiliary Task! (at least equal)
	BEST. Val. Acc Aux: 0.649
---------------------------------------

[Epoch: 2][#examples: 5888/46620][#steps: 1550]
	Train Loss: 0.298 | Train PPL:   1.347 | lr: 3.000e-03
[Epoch: 2][#examples: 9088/46620][#steps: 1600]
	Train Loss: 0.297 | Train PPL:   1.346 | lr: 3.000e-03
[Epoch: 2][#examples: 12288/46620][#steps: 1650]
	Train Loss: 0.301 | Train PPL:   1.351 | lr: 3.000e-03
[Epoch: 2][#examples: 15488/

[Epoch: 4][#examples: 37376/46620][#steps: 3500]
	Train Loss: 0.221 | Train PPL:   1.247 | lr: 2.700e-03
-----Val------
[VAL]: The number of correct predictions (main-task (multi)): 3979/5828
[VAL]: The number of correct predictions (aux-task (multi)): 3964/5828

---------------------------------------
[Epoch: 4][Validatiing...]
		 Better Valid Acc! (at least equal)
	 Early Stopping Patience: 99/100
	 Val. Loss: 0.563 | Val. Acc: 0.683 | Val. PPL:   1.755
	 BEST. Val. Loss: 0.552 | BEST. Val. Acc: 0.683 | Val. Loss: 0.563 | BEST. Val. Epoch: 4 | BEST. Val. Step: 3500
---------------------------------------

		 Better Valid Acc on Auxiliary Task! (at least equal)
	BEST. Val. Acc Aux: 0.680
---------------------------------------

[Epoch: 4][#examples: 40576/46620][#steps: 3550]
	Train Loss: 0.222 | Train PPL:   1.249 | lr: 2.700e-03
[Epoch: 4][#examples: 43776/46620][#steps: 3600]
	Train Loss: 0.223 | Train PPL:   1.250 | lr: 2.700e-03
[VAL]: The number of correct predictions (main-task

[Epoch: 7][#examples: 6208/46620][#steps: 5200]
	Train Loss: 0.093 | Train PPL:   1.098 | lr: 1.771e-03
[Epoch: 7][#examples: 9408/46620][#steps: 5250]
	Train Loss: 0.092 | Train PPL:   1.097 | lr: 1.771e-03
[Epoch: 7][#examples: 12608/46620][#steps: 5300]
	Train Loss: 0.097 | Train PPL:   1.102 | lr: 1.771e-03
[Epoch: 7][#examples: 15808/46620][#steps: 5350]
	Train Loss: 0.097 | Train PPL:   1.102 | lr: 1.771e-03
[Epoch: 7][#examples: 19008/46620][#steps: 5400]
	Train Loss: 0.097 | Train PPL:   1.102 | lr: 1.771e-03
[Epoch: 7][#examples: 22208/46620][#steps: 5450]
	Train Loss: 0.097 | Train PPL:   1.102 | lr: 1.771e-03
[Epoch: 7][#examples: 25408/46620][#steps: 5500]
	Train Loss: 0.098 | Train PPL:   1.103 | lr: 1.771e-03
-----Val------
[VAL]: The number of correct predictions (main-task (multi)): 4085/5828
[VAL]: The number of correct predictions (aux-task (multi)): 4120/5828

---------------------------------------
[Epoch: 7][Validatiing...]
		 Better Valid Acc! (at least equal)
	 E

[VAL]: The number of correct predictions (main-task (multi)): 4126/5828
[VAL]: The number of correct predictions (aux-task (multi)): 4147/5828

---------------------------------------
[Epoch: 9][Validatiing...]
		 Better Valid Acc! (at least equal)
	 Early Stopping Patience: 91/100
	 Val. Loss: 0.591 | Val. Acc: 0.708 | Val. PPL:   1.805
	 BEST. Val. Loss: 0.541 | BEST. Val. Acc: 0.708 | Val. Loss: 0.591 | BEST. Val. Epoch: 9 | BEST. Val. Step: 7290
---------------------------------------

	BEST. Val. Acc Aux: 0.716
---------------------------------------

Epoch: 10 | Time: 1m 27s
	Train Loss: 0.059 | Train PPL:   1.061
	 Val. Loss: 0.591 | Val. Acc: 0.708 | Val. PPL:   1.805
[Train]: Current Teacher Forcing Ratio: 0.500
[Epoch: 10][#examples: 640/46620][#steps: 7300]
	Train Loss: 0.042 | Train PPL:   1.043 | lr: 1.291e-03
[Epoch: 10][#examples: 3840/46620][#steps: 7350]
	Train Loss: 0.042 | Train PPL:   1.042 | lr: 1.291e-03
[Epoch: 10][#examples: 7040/46620][#steps: 7400]
	Train Loss

...

[Epoch: 36][#examples: 21120/81252][#steps: 46050]
	Train Loss: 0.020 | Train PPL:   1.020 | lr: 9.838e-08
[Epoch: 36][#examples: 24320/81252][#steps: 46100]
	Train Loss: 0.020 | Train PPL:   1.020 | lr: 9.838e-08
[Epoch: 36][#examples: 27520/81252][#steps: 46150]
	Train Loss: 0.020 | Train PPL:   1.020 | lr: 9.838e-08
[Epoch: 36][#examples: 30720/81252][#steps: 46200]
	Train Loss: 0.020 | Train PPL:   1.020 | lr: 9.838e-08
[Epoch: 36][#examples: 33920/81252][#steps: 46250]
	Train Loss: 0.020 | Train PPL:   1.020 | lr: 9.838e-08
[Epoch: 36][#examples: 37120/81252][#steps: 46300]
	Train Loss: 0.019 | Train PPL:   1.020 | lr: 9.838e-08
[Epoch: 36][#examples: 40320/81252][#steps: 46350]
	Train Loss: 0.019 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 36][#examples: 43520/81252][#steps: 46400]
	Train Loss: 0.019 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 36][#examples: 46720/81252][#steps: 46450]
	Train Loss: 0.019 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 36][#examples: 49920/81252][#

[Epoch: 38][#examples: 2560/81252][#steps: 48300]
	Train Loss: 0.017 | Train PPL:   1.017 | lr: 9.838e-08
[Epoch: 38][#examples: 5760/81252][#steps: 48350]
	Train Loss: 0.019 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 38][#examples: 8960/81252][#steps: 48400]
	Train Loss: 0.020 | Train PPL:   1.020 | lr: 9.838e-08
[Epoch: 38][#examples: 12160/81252][#steps: 48450]
	Train Loss: 0.019 | Train PPL:   1.020 | lr: 9.838e-08
[Epoch: 38][#examples: 15360/81252][#steps: 48500]
	Train Loss: 0.018 | Train PPL:   1.018 | lr: 9.838e-08
-----Val------
[VAL]: The number of correct predictions (main-task (multi)): 340/513
[VAL]: The number of correct predictions (aux-task (multi)): 341/513

---------------------------------------
[Epoch: 38][Validatiing...]
	 Early Stopping Patience: 19/100
	 Val. Loss: 1.698 | Val. Acc: 0.663 | Val. PPL:   5.463
	 BEST. Val. Loss: 1.680 | BEST. Val. Acc: 0.665 | Val. Loss: 1.689 | BEST. Val. Epoch: 17 | BEST. Val. Step: 22500
--------------------------------------

[Epoch: 39][#examples: 65280/81252][#steps: 50550]
	Train Loss: 0.018 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 39][#examples: 68480/81252][#steps: 50600]
	Train Loss: 0.018 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 39][#examples: 71680/81252][#steps: 50650]
	Train Loss: 0.018 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 39][#examples: 74880/81252][#steps: 50700]
	Train Loss: 0.018 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 39][#examples: 78080/81252][#steps: 50750]
	Train Loss: 0.019 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 39][#examples: 81280/81252][#steps: 50800]
	Train Loss: 0.019 | Train PPL:   1.019 | lr: 9.838e-08
[VAL]: The number of correct predictions (main-task (multi)): 340/513
[VAL]: The number of correct predictions (aux-task (multi)): 341/513

---------------------------------------
[Epoch: 39][Validatiing...]
	 Early Stopping Patience: 13/100
	 Val. Loss: 1.698 | Val. Acc: 0.663 | Val. PPL:   5.463
	 BEST. Val. Loss: 1.680 | BEST. Val. Acc: 0.665 | Val. Los

[Epoch: 41][#examples: 43520/81252][#steps: 52750]
	Train Loss: 0.018 | Train PPL:   1.018 | lr: 9.838e-08
[Epoch: 41][#examples: 46720/81252][#steps: 52800]
	Train Loss: 0.018 | Train PPL:   1.018 | lr: 9.838e-08
[Epoch: 41][#examples: 49920/81252][#steps: 52850]
	Train Loss: 0.018 | Train PPL:   1.018 | lr: 9.838e-08
[Epoch: 41][#examples: 53120/81252][#steps: 52900]
	Train Loss: 0.019 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 41][#examples: 56320/81252][#steps: 52950]
	Train Loss: 0.019 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 41][#examples: 59520/81252][#steps: 53000]
	Train Loss: 0.019 | Train PPL:   1.019 | lr: 9.838e-08
-----Val------
[VAL]: The number of correct predictions (main-task (multi)): 340/513
[VAL]: The number of correct predictions (aux-task (multi)): 341/513

---------------------------------------
[Epoch: 41][Validatiing...]
	 Early Stopping Patience: 7/100
	 Val. Loss: 1.698 | Val. Acc: 0.663 | Val. PPL:   5.463
	 BEST. Val. Loss: 1.680 | BEST. Val. Acc: 0.

[Epoch: 43][#examples: 21760/81252][#steps: 54950]
	Train Loss: 0.019 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 43][#examples: 24960/81252][#steps: 55000]
	Train Loss: 0.019 | Train PPL:   1.019 | lr: 9.838e-08
-----Val------
[VAL]: The number of correct predictions (main-task (multi)): 340/513
[VAL]: The number of correct predictions (aux-task (multi)): 341/513

---------------------------------------
[Epoch: 43][Validatiing...]
	 Early Stopping Patience: 1/100
	 Val. Loss: 1.698 | Val. Acc: 0.663 | Val. PPL:   5.463
	 BEST. Val. Loss: 1.680 | BEST. Val. Acc: 0.665 | Val. Loss: 1.689 | BEST. Val. Epoch: 17 | BEST. Val. Step: 22500
---------------------------------------

	BEST. Val. Acc Aux: 0.684
---------------------------------------

[Epoch: 43][#examples: 28160/81252][#steps: 55050]
	Train Loss: 0.019 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 43][#examples: 31360/81252][#steps: 55100]
	Train Loss: 0.019 | Train PPL:   1.019 | lr: 9.838e-08
[Epoch: 43][#examples: 34560/81252